# Process stackoverflow data

In [69]:
%load_ext autoreload
%autoreload 2

from google.cloud import bigquery
import ast
import glob
import re
from pathlib import Path

import astor
import pandas as pd
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

from general_utils import apply_parallel, flattenlist
from bs4 import BeautifulSoup

EN = spacy.load('en')
pd.set_option('display.max_colwidth', -1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
use_cache = False

In [71]:
# TODO: use csv for now, fix credential issue
# bigquery
# https://console.cloud.google.com/bigquery?sq=784649123063:df10117d54ab4ef2a853ce5ba8e583dc

# if use_cache == False: 
#     # get data from bigquery
#     client = bigquery.Client()
#     sql = """
#     SELECT id, body, last_activity_date, score, text as comment
#     FROM `bigquery-public-data.stackoverflow.posts_answers` as answer
#     JOIN (SELECT post_id, text from `bigquery-public-data.stackoverflow.comments`) as c on c.post_id = answer.id
#     WHERE 
#     ( Date(last_activity_date) >= Date("2015-01-01")
#       AND 
#       answer.parent_id IN (
#         SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions`
#         WHERE LOWER(tags) LIKE '%python%'
#       ))

#     """
#     df = client.query(sql).to_dataframe()
#     df.to_csv("data/processed_data/stackoverflow/raw.csv")

# Read from csv files

In [72]:
%%time
# read from csv
df = pd.read_csv("data/stackoverflow/processed_data/raw9756.csv")

CPU times: user 118 ms, sys: 4.57 ms, total: 123 ms
Wall time: 122 ms


In [73]:
df.head()

,id,body,last_activity_date,score,comment
0,42182911,"<p>Well, since its a regular expression you could just escape it.</p>\n\n<pre><code>Stat: {&lt;CD&gt;&lt;\%&gt;(&lt;NN&gt;+|&lt;VBN&gt;|JJ)?}\n</code></pre>\n\n<p>You could also have a list of keywords you want to replace, so that your chunk rules don't become excessively long.</p>\n\n<p>e.g.</p>\n\n<pre><code>s = '56% rise and 75% fall'\ngen_replacements = [('%', 'PERCENTAGE'), ('perc.', 'PERCENTAGE'), etc]\nfor ndl, rpl in gen_replacements:\n s = s.replace(' %s ' % ndl, ' %s ' % rpl)\n\nStat: {&lt;CD&gt;&lt;PERCENTAGE&gt;(&lt;NN&gt;+|&lt;VBN&gt;|JJ)?}\n</code></pre>",2017-02-12 00:32:44.407 UTC,0,"They're not mere regexes – otherwise, `<NN>+` would be interpreted as *""one lower-than, two N's, and one or more greater-than's""*, instead of *""one or more nouns""*. There is an additional translation step. And they are applied to the sequence of tags, not the text itself."
1,46213700,"<p><a href=""https://github.com/django/django/blob/1.11/django/contrib/auth/views.py#L94"" rel=""nofollow noreferrer"">Django isn't checking <code>next</code> in <code>extra_context</code></a> but in <code>GET</code> and <code>POST</code> params. </p>\n\n<p><a href=""https://github.com/django/django/blob/1.11/django/contrib/auth/views.py#L129"" rel=""nofollow noreferrer""><code>extra_context</code> is used to update your template context</a>. So if you want to pass the values for variables to your template you can set those with <code>extra_context</code>.</p>\n\n<p>However, you can fix your test by either <a href=""https://github.com/django/django/blob/1.11/django/contrib/auth/views.py#L104"" rel=""nofollow noreferrer"">setting <code>LOGIN_REDIRECT_URL</code> in <code>settings.py</code></a> or passing next as a POST or GET param.</p>\n\n<pre><code>params = dict(**self.credentials, next=reverse(""home:index""))\nresponse = self.client.post(\n url,\n params,\n follow=True\n)\n</code></pre>\n\n<h3>NOTE:</h3>\n\n<blockquote>\n <p><code>self.credentials</code> should be unpacked in the dict being applied to <code>self.data</code>.</p>\n</blockquote>\n\n<pre><code>self.data = dict(\n **self.credentials,\n next=reverse(""administration:dashboard"")\n)\n</code></pre>\n\n<p>If you wanted to test without making this change, I can recommend another way to test this.</p>\n\n<p>You need to render your template in a browser and fill out the login form. This way your browser client will make the <code>POST</code> request with <code>next</code> passed as a parameter.\nYou can get this done with a headless browser using <code>selenium webdriver</code>.</p>",2017-09-14 17:16:22.787 UTC,0,See my note about unpacking `self.credentials` in the `dict` assigned to self.data
2,51806306,"<p>You can use font attribute of worksheet column, something like this:</p>\n\n<pre><code>&gt;&gt;&gt; my_col = sheet.column_dimensions['A']\n&gt;&gt;&gt; my_col.font = italic24Font\n</code></pre>\n\n<p>But, note that this applies only to cells created (in Excel). If you want to apply styles to entire column then you must apply the style to each cell by iteration.</p>",2018-08-12 06:25:22.257 UTC,0,"Hello, thanks for replying! After I run the code, I opened the file test.xlsx and the whole of column A seems to have disappeared. What went wrong"
3,51159568,"<p>You could just keep checking each future in the sequence till you find one that is done.</p>\n\n<pre><code>from collections import deque\npool = concurrent.futures.ThreadPoolExecutor(max_workers = 4)\n\nfutures = deque(pool.submit(g,i) for i in range (4))\nj = 4\nresult = False\nwhile not result:\n while not futures[0].done():\n futures.rotate()\n future = futures.popleft()\n result = future.result()\n if not result:\n j += 1\n futures.append(pool.submit(g,j))\n</code></pre>\n\n<hr>\n\n<p>Here is a similar solution using <code>concurrent.futures.wait</code>.<br>\nFirst a callable for testing purposes:</p>\n\n<pre><code>import random\nclass F:\n def __init__(self, threshold=.03):\

In [74]:
df.shape

(9756, 5)

In [75]:
def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]


def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)


# Aggregate comments

In [76]:
# concat comments for each post

grouped = df.groupby(['id', 'body','last_activity_date','score'],as_index=False).comment.agg(lambda x: "\n".join(x))
deduped_df = pd.DataFrame(grouped)

In [77]:
deduped_df.head(1)

,id,body,last_activity_date,score,comment
0,663054,"<p><code>close_fds</code> <em><a href=""http://docs.python.org/library/subprocess.html#subprocess.Popen"" rel=""nofollow noreferrer""><strong>is</strong> supported on Windows</a></em> (search for ""close_fds"" after that link) starting with Python 2.6 (if <code>stdin</code>/<code>stdout</code>/<code>stderr</code> are not redirected). You might consider upgrading.</p>\n\n<p><strong>UPDATE 2017-11-16 after a down vote (why?)</strong>: From the linked doc:</p>\n\n<blockquote>\n <p>Note that on Windows, you cannot set close_fds to true and also\n redirect the standard handles by setting stdin, stdout or stderr.</p>\n</blockquote>\n\n<p>So you can either <code>subprocess.call</code> with <code>close_fds = True</code> and not setting <code>stdin</code>, <code>stdout</code> or <code>stderr</code> (the default) (or setting them to <code>None</code>):</p>\n\n<blockquote>\n <p><code>subprocess.call(command, shell=True, close_fds = True)</code></p>\n</blockquote>\n\n<p>or you <code>subprocess.call</code> with <code>close_fds = False</code>:</p>\n\n<blockquote>\n <p><code>subprocess.call(command, shell=True, stderr=errptr, stdout=outptr, close_fds = False)</code></p>\n</blockquote>\n\n<p>or (Python >= 3.2) you let <code>subprocess.call</code> figure out the value of <code>close_fds</code> by itself:</p>\n\n<blockquote>\n <p><code>subprocess.call(command, shell=True, stderr=errptr, stdout=outptr)</code></p>\n</blockquote>",2017-11-16 09:48:40.917 UTC,0,"@vaab See my update, where I'm trying to address this"


In [78]:
deduped_df.shape

(8359, 5)

# Extract text from post body

In [165]:
%%time

content_list = []
url_list = []

for i, row in deduped_df.iterrows():
    content = row.body
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
    if text =='': 
        continue
    content_list.append(' '.join(tokenize_text(text)))
    url_list.append('https://stackoverflow.com/a/' + str(row.id))


content_token_df = pd.DataFrame({'content_token': content_list, 'url': url_list})

CPU times: user 15.4 s, sys: 0 ns, total: 15.4 s
Wall time: 15.4 s


In [168]:
content_token_df.head()

,content_token,url
0,"is supported on windows ( search for "" close_fds "" after that link ) starting with python 2.6 ( if stdin / stdout / stderr are not redirected ) . you might consider upgrading .",https://stackoverflow.com/a/663054
1,this is because redirecting to a view misses the original context you use to render the page in the first place .,https://stackoverflow.com/a/1129743
2,"you can use aes to encrypt your string with a password . though , you 'll want to chose a strong enough password so people ca n't easily guess what it is ( sorry i ca n't help it . i 'm a wannabe security weenie ) .",https://stackoverflow.com/a/2490354
3,"you could install pylint using and use it via notepad++ 's run ... command ( f5):c:\python34\scripts\pylint.bat "" $ ( full_current_path ) """,https://stackoverflow.com/a/5350204
4,have you tried removing :,https://stackoverflow.com/a/6740499


In [169]:
content_token_df.shape

(8190, 2)

# Partition train/test/validate data

In [226]:
# train, valid, test splits
train, test = train_test_split(list(content_token_df.groupby('url')), train_size=0.87, shuffle=True, random_state=8081)
train, valid = train_test_split(train, train_size=0.82, random_state=8081)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [227]:
train = pd.concat([d for _, d in train]).reset_index(drop=True)
test = pd.concat([d for _, d in test]).reset_index(drop=True)
valid = pd.concat([d for _, d in valid]).reset_index(drop=True)

In [228]:
train.head()

,content_token,url
0,"please add "" html5lib "" or "" html.parser """,https://stackoverflow.com/a/43595599
1,"see the sample provided in apps script documentation,// logs information about any attachments in the first 100 inbox threads . var threads = gmailapp.getinboxthreads(0 , 100 ) ; var msgs = gmailapp.getmessagesforthreads(threads ) ; for ( var i = 0 ; i < msgs.length ; i++ ) { for ( var j = 0 ; j < msgs[i].length ; j++ ) { var attachments = msgs[i][j].getattachments ( ) ; for ( var k = 0 ; k < attachments.length ; k++ ) { logger.log('message "" % s "" contains the attachment "" % s "" ( % s bytes ) ' , msgs[i][j].getsubject ( ) , attachments[k].getname ( ) , attachments[k].getsize ( ) ) ; } } }",https://stackoverflow.com/a/51473194
2,"this not something i have tried , but i have found a paper , which states "" abaqus does not offer the geometric stiffness matrix as a standard output "" .",https://stackoverflow.com/a/43365314
3,"you also can inherit from autofield and avoid the call to by overriding the check method.class myautofield(models . autofield ) : def check(self , * * kwargs ) : errors = models . field.check(**kwargs ) # errors.extend(self._check_primary_key ( ) ) return errors class mymodel(models . model ) : uid = models . uuidfield(primary_key = true , default = uuid.uuid4 , editable = false ) cod = myautofield(editable = false , unique = true )",https://stackoverflow.com/a/41769893
4,"a .apply(lambda ... ) is a nice paradigm in pandas that will go over every row and perform a function . here i 'm making a list out of space - delimitated words in the description column via .split ( ) then converting it back into a string with a , separation via .join ( )",https://stackoverflow.com/a/50648425


In [229]:
test.head()

,content_token,url
0,"if the input comes in one line , you could try to it . m , n = input().split ( ) m = int(m ) n = int(n ) k = m*n if k%2 ! = 0 : print('akshat ' ) else : print('malvika ' )",https://stackoverflow.com/a/47149069
1,you can do it with the ticktext input,https://stackoverflow.com/a/39621727
2,the first will always be executed according to this . so the if is not necessary .,https://stackoverflow.com/a/29797419
3,"as with everything in beautifulsoup , you have many alternatives . for value pairs i tend to use function most : from bs4 import beautifulsoup html = ' ' ' \ < tr > < td > project id:</td > < td > < span>112120</span></td > < /tr > ' '' soup = beautifulsoup(html , ' lxml ' ) # option 1 td = soup.select('td ' ) for name , value in zip(td , td[1 : ] ) : print(name.text , value.text ) # option 2 td = soup.find(lambda t : t.text.strip()=='project id:').parent.select('td ' ) for name , value in zip(td , td[1 : ] ) : print(name.text , value.text ) # option 3 td1 = soup.find(lambda t : t.text.strip()=='project id : ' ) td2 = td1.find_next('td ' ) print(td1.text , td2.text )",https://stackoverflow.com/a/51641874
4,something like this :,https://stackoverflow.com/a/45313084


In [230]:
valid.head()

,content_token,url
0,"you can use module . > > > import datetime > > > do = datetime.datetime.strptime(""dec 17 2014 00:00:06 "" , "" % b % d % y % h:%m:%s "" ) > > > do.strftime(""%y-%m-%d % h:%m:%s "" ) ' 2014 - 12 - 17 00:00:06 '",https://stackoverflow.com/a/27900759
1,you could do something like this to get a list of all indices of the letter,https://stackoverflow.com/a/42908260
2,you can use this method : spreadsheets.create to create a spreadsheet .,https://stackoverflow.com/a/46687600
3,understood your question now :,https://stackoverflow.com/a/46901010
4,here 's a solution for what i understand the format of your data to be . please confirm exact format if i 've misunderstood .,https://stackoverflow.com/a/48995277


In [231]:
assert valid.shape[0] + test.shape[0] + train.shape[0] == content_token_df.shape[0],f'Row count mismatch.'

# Write to file

In [232]:
def write_to(df, filename, path='./data/stackoverflow/processed_data'):
    "Helper function to write processed files to disk."
    out = Path(path)
    out.mkdir(exist_ok=True)
    df.content_token.to_csv(out/'{}.content_token'.format(filename), index=False)
    df.url.to_csv(out/'{}.url'.format(filename), index=False)

In [233]:
write_to(train, 'train')
write_to(valid, 'valid')
write_to(test, 'test')

In [235]:
!ls -lah ./data/stackoverflow/processed_data

total 23M
drwxr-xr-x 3 root root 4.0K Nov 24 15:33 .
drwxr-xr-x 4 5002 5002 4.0K Nov 20 08:14 ..
drwxr-xr-x 2 root root 4.0K Nov 20 08:26 .ipynb_checkpoints
-rw-r--r-- 1 root root  11M Nov 20 08:27 raw.csv
-rw-r--r-- 1 root root  11M Nov 20 08:25 raw9756.csv
-rw-r--r-- 1 root root 237K Nov 24 15:33 test.content_token
-rw-r--r-- 1 root root  39K Nov 24 15:33 test.url
-rw-r--r-- 1 root root 1.3M Nov 24 15:33 train.content_token
-rw-r--r-- 1 root root 212K Nov 24 15:33 train.url
-rw-r--r-- 1 root root 255K Nov 24 15:33 valid.content_token
-rw-r--r-- 1 root root  47K Nov 24 15:33 valid.url
